<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Sentence-embedding" data-toc-modified-id="Sentence-embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sentence embedding</a></span><ul class="toc-item"><li><span><a href="#Infersent" data-toc-modified-id="Infersent-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Infersent</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#TSNE" data-toc-modified-id="TSNE-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>TSNE</a></span></li></ul></li><li><span><a href="#Blocking" data-toc-modified-id="Blocking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Blocking</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#bilstm,-rnn_dim-2048" data-toc-modified-id="bilstm,-rnn_dim-2048-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>bilstm, rnn_dim 2048</a></span></li><li><span><a href="#Searching" data-toc-modified-id="Searching-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Searching</a></span></li></ul></li><li><span><a href="#Best" data-toc-modified-id="Best-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Best</a></span></li></ul></div>

# Setup 

In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
params = {}

# Import dataset

In [2]:
from preprocessing_datasets.preprocessing_amzn_gp import clean_amzn_gp
table, pairs = clean_amzn_gp()

In [3]:
table

,title,description,manufacturer,price
0,clickart 950 000 - premier image pack (dvd-rom),unk,broderbund,0
1,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0
2,noah's ark activity center (jewel case ages 3-8),unk,victory multimedia,0
3,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99
...,...,...,...,...
4584,jumpstart(r) advanced 1st grade,prepare your child for the 1st grade and beyon...,unk,19.99
4585,ibm(r) viavoice(r) advanced edition 10,ibm viavoice advanced edition release 10 is a ...,unk,78.95
4586,xbox 360: gears of war,as marcus fenix you fight a war against the im...,unk,59.99
4587,documents to go premium 7.0,this pda software enables you to use your docu...,unk,49.99


# Sentence embedding

## Infersent

Setup model

In [4]:
from embedding_algorithms.inferSent import set_RNN_embedding
start = time.time()
model_type = "bilstm" 
char_level = False
model_version = 2
rnn_dim = 2048
verbose = 1
set_RNN_embedding(model_type, char_level, model_version, rnn_dim, verbose)
print("TIME: {0}".format(time.time() - start))

params['embedding'] = {
    'name': 'Infersent',
    'model_type': model_type,
    'char_level': char_level,
    'model_version': model_version,
    'rnn_dim': rnn_dim,
    'verbose': verbose
}

Vocab size : 2196017
TIME: 165.48254489898682


Embedding

In [5]:
from embedding_algorithms.inferSent import RNN_embedding
start = time.time()
attr_list = ['title', 'manufacturer']
params['attr_list'] = attr_list
embeddings_tokens = RNN_embedding(table, attr_list, model_type, char_level)
embeddings_tokens = np.array(embeddings_tokens)
print("TIME: {0}".format(time.time() - start))

attrs: ['title', 'manufacturer']
TIME: 77.1130301952362


# Dimensionality reduction

## TSNE

In [6]:
from dimensionality_reduction_algorithms.tsne import tsne_dim_reduction

start = time.time()

tsne_embeddings = tsne_dim_reduction(
    embeddings_tokens, 
    num_components=2,
    early_exaggeration=12,
    verbose=1,
    perplexity=5,
    method="barnes_hut")

print("TIME: {0}".format(time.time() - start))
params['reduction'] = {
    'name': 'TSNE',
    'num_components': 2,
    'early_exaggeration': 12,
    'verbose': 1,
    'perplexity': 5,
    'method': "barnes_hut"
}

starting dimension: 4096
setting TSNE with n_components: 2 & perplexity: 5
early_exaggeration: 12
TIME: 237.0320119857788


# Blocking

In [16]:
from cluster_algorithms.kMeans_cluster import kMean_cluster
num_clusters = 13

start = time.time()
blocks = kMean_cluster(tsne_embeddings, {'num_clusters': num_clusters, 'distance_algorithm': 'euclidean'})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'k_means',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 13, distance_algorithm = <function euclidean_distance at 0x7fe5be9ea5f0>
BLOCKS: 163.54882884025574


In [13]:
from cluster_algorithms.birch_cluster import birch_cluster
num_clusters = 28

start = time.time()
blocks = birch_cluster(tsne_embeddings, {'num_clusters': num_clusters})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'birch_cluster',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 28, 
BLOCKS: 0.6017839908599854


In [14]:
from cluster_algorithms.DBScan_cluster import DBSCAN_cluster
eps = 7
min_samples = 2

start = time.time()
blocks = DBSCAN_cluster(tsne_embeddings, {'eps':eps, 'min_samples':min_samples})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'DBScan',
    'eps': eps,
    'min_samples': min_samples
}

DBScan_cluster
BLOCKS: 0.039164066314697266


In [8]:
from cluster_algorithms.hierarchy_cluster import hierarchy_cluster
num_clusters = 15
start = time.time()
blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'hierarchy',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 15, 
BLOCKS: 0.7234680652618408


# Evaluation

## bilstm, rnn_dim 2048

In [9]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9277423762482704
(PC) Pair completeness is: 0.8407692307692308
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8821171893191397
(PQ) Pair quality - Precision is: 0.0014368949397108857
(FM) Fmeasure is: 0.0028688868823887616

['title', 'manufacturer']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 5, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 15}


In [14]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking']) 

(RR) Reduction ratio is: 0.9613279585407887
(PC) Pair completeness is: 0.7553846153846154
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8460034151906496
(PQ) Pair quality - Precision is: 0.002412142262353632
(FM) Fmeasure is: 0.0048089283484367305

['title', 'manufacturer']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 5, 'method': 'barnes_hut'}
{'name': 'birch_cluster', 'num_clusters': 28}


In [15]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9603648313325732
(PC) Pair completeness is: 0.7276923076923076
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8279934182116965
(PQ) Pair quality - Precision is: 0.0022672476189106667
(FM) Fmeasure is: 0.004520411137604947

['title', 'manufacturer']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 5, 'method': 'barnes_hut'}
{'name': 'DBScan', 'eps': 7, 'min_samples': 2}


In [17]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9219618081447561
(PC) Pair completeness is: 0.7523076923076923
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8285392048218918
(PQ) Pair quality - Precision is: 0.0011904747413638847
(FM) Fmeasure is: 0.0023771877480035144

['title', 'manufacturer']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 5, 'method': 'barnes_hut'}
{'name': 'k_means', 'num_clusters': 13}


## Searching

In [12]:
for i in range(5,50,1):
    num_clusters = int(i)
    blocks = birch_cluster(tsne_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

clustering with NUM_CLUSTERS = 5, 
(RR) Reduction ratio is: 0.7739433385965415
(PC) Pair completeness is: 0.8223076923076923
(RM) Reference metric (Harmonic mean RR and PC) is: 0.7973928265878308
(PQ) Pair quality - Precision is: 0.0004492094921453472
(FM) Fmeasure is: 0.0008979284647523179
clustering with NUM_CLUSTERS = 6, 
(RR) Reduction ratio is: 0.7941405122708238
(PC) Pair completeness is: 0.8192307692307692
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8064905458580295
(PQ) Pair quality - Precision is: 0.0004914363183898239
(FM) Fmeasure is: 0.0009822833892189557
clustering with NUM_CLUSTERS = 7, 
(RR) Reduction ratio is: 0.8459766854631151
(PC) Pair completeness is: 0.8046153846153846
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8247778097191272
(PQ) Pair quality - Precision is: 0.0006451099616449441
(FM) Fmeasure is: 0.0012891863028269045
clustering with NUM_CLUSTERS = 8, 
(RR) Reduction ratio is: 0.86110896322904
(PC) Pair completeness is: 0.7861538461538462
(

(RR) Reduction ratio is: 0.9680906523180123
(PC) Pair completeness is: 0.7438461538461538
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8412816474396341
(PQ) Pair quality - Precision is: 0.0028787044341574506
(FM) Fmeasure is: 0.005735213439497057
clustering with NUM_CLUSTERS = 35, 
(RR) Reduction ratio is: 0.9688486910912205
(PC) Pair completeness is: 0.7423076923076923
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8405822437464415
(PQ) Pair quality - Precision is: 0.002942656319087624
(FM) Fmeasure is: 0.005862074202317494
clustering with NUM_CLUSTERS = 36, 
(RR) Reduction ratio is: 0.9696702797315061
(PC) Pair completeness is: 0.7423076923076923
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8408913191674734
(PQ) Pair quality - Precision is: 0.0030223686600727873
(FM) Fmeasure is: 0.0060202254621224876
clustering with NUM_CLUSTERS = 37, 
(RR) Reduction ratio is: 0.9701666146425353
(PC) Pair completeness is: 0.74
(RM) Reference metric (Harmonic mean RR and PC) 

In [13]:
for i in range(6,8,1):
    eps = int(i)
    min_samples = 2
    print(eps)
    blocks = DBSCAN_cluster(tsne_embeddings, {'eps':eps, 'min_samples':min_samples})
    calc_index(blocks,table,pairs)

6
DBScan_cluster
(RR) Reduction ratio is: 0.9864605535810873
(PC) Pair completeness is: 0.7
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8189013209238515
(PQ) Pair quality - Precision is: 0.0063845311929952575
(FM) Fmeasure is: 0.012653651482284888
7
DBScan_cluster
(RR) Reduction ratio is: 0.9603648313325732
(PC) Pair completeness is: 0.7276923076923076
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8279934182116965
(PQ) Pair quality - Precision is: 0.0022672476189106667
(FM) Fmeasure is: 0.004520411137604947


# Best

In [28]:
# Amazon-GP
# RR PC alpha
list_embedding = [0.9436, 0.7885, 0.8591]

list_hierarchy = [0.9277, 0.8407, 0.8821] # TSNE: 203.28, BLOCKS: 0.7234, n_cluster: 15
list_kmeans = [0.9219, 0.7523, 0.8285] # TSNE: 203.28, BLOCKS: 163.54, n_cluster: 13
list_birch = [0.9613, 0.7553, 0.8460] # TSNE: 203.28, BLOCKS: 0.601 n_cluster: 26
list_DBscan = [0.9603, 0.7276, 0.8276] # TSNE: 203.28, BLOCKS: 0.7234